<a href="https://colab.research.google.com/github/fatday/CME241_RLFinance/blob/main/student_copy_of_cme_241_2025_exam_official_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stanford CME 241 (Winter 2025) - Exam

## Instructions

**We recommend taking this exam using Google Colab. The first step to do right now is to copy this notebook by clicking "File → Save A Copy in Drive" at the top left. You will be able to edit the copy that opens in a new tab.**

Welcome to the exam for *Foundations of Reinforcement Learning with Applications in Finance*. Here are instructions for taking the exam:

- There is a 72 hour exam period in which you can take the exam. We only expect the exam to take about 3-6 hours, but we want to allow extra time to minimize stress. The Exam Period begins Friday, Feb 28th at 8:30 PM PST and ends at the deadline: Monday, March 3rd at 8:30 PM PST. You must submit your exam by the deadline.
- There are 2 problems worth a total of 30 points. You can use the Table of Contents on the left navigation bar (click the icon that looks like a bulleted list) to quickly move between the problems and examine the number of points for each.
- Put all of your writing, math formulas, code, graphs etc. in your copy of this Google Colab.
- Show work! Explain your thoughts! This helps with partial credit.
- To submit, print your copy of this Google Colab to PDF using File -> Print. Submit the PDF to the `Exam` assignment on Gradescope. Finally, share your copy of this Google Colab (via the "Share" button at the upper right) with `neelsn@stanford.edu`.
- Do not share or upload this exam or any documents derived from it to any public platform.
- Please use your time wisely and balance with the appropriate number of points in the questions.
- You will be graded mainly on correctness, though partial credit will be awarded so please explain your process and intutition where appropriate. Where code is expected, please write code that is easy to read and understand (i.e. well-commented and typed).
- If you have any trouble during the exam, please email `neelsn@stanford.edu`.
- Ed will be disabled during the 72 hour exam period. Send any questions/clarifications to `neelsn@stanford.edu`.

## Honor Code

**Please note the following:**

- We trust that you will follow [The Stanford Honor Code](https://communitystandards.stanford.edu/policies-and-guidance/honor-code).
- Consulting with any other humans, students or otherwise, is <ins>NOT</ins> allowed. This exam is to be taken individually.
- ChatGPT/Claude/Gemini/any other LLMs are <ins>NOT</ins> allowed. We have run each of these problems through many LLMs and know what answers they produce. If you do not know the answer, try your best; do not use LLMs for this exam in any capacity. Again, we can tell when they have been used. Also, as a word of caution, LLMs have gotten these problems wrong across many different prompts; the exam was designed in a manner that would trip up LLMs, hence the faulty answers provided by them.

Inputting your information below acts as acknowledgement of your adherence to the Stanford Honor Code and the rules of this exam. Failure to comply will result in disciplinary action and a 0 on the exam.

**Name:** Put Your Name Here

**SUNet ID:** Put Your SUNET ID Here

**Colab URL:** Put the URL to this Colab Here

## Code Setup

There are a few steps to take to set up this notebook.

First, connect to a hosted runtime by clicking the Connect tab in the top right and proceed with the exam. Once you do this, you should see a green checkmark with the words "RAM" and "Disk" next to it. If you already see this, you're already connected, and can skip this step. (Note that there is a small chance runtimes disconnect in the middle of the 4-hour exam. If so, you should be able to connect again by clicking "Reconnect". You may then click on the menu item "Runtime → Run Before" to rerun all cells and recover progress.)

This exam is setup as a self-contained Colab/Jupyter notebook that clones the RL book git repository. All libraries within RL book as well as the external libraries that are required are automatically downloaded in this Jupyter notebook.

The following shell script command (with the ! prefix) enables this. Make sure you run this cell just one time, as it takes a minute or so to get all libraries downloaded. After you have run the cell, the rest of the jupyter notebook can be run repeatedly as you make your changes (avoid running this cell a second time so your repeated runs are quick enough).

If you have issues running it, immediately reach out to `neelsn@stanford.edu`.

In [ ]:
# Clone the rl-book repository
!git clone https://github.com/TikhonJelvis/rl-book.git

# Change the working directory to the rl-book directory
%cd rl-book

# Move to the branch with proper installation requirements
!git checkout notebook
!pip install -r notebooks/notebook-requirements.txt
!pip install -e .

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple, Callable, Iterator
import numpy as np
from math import comb
from dataclasses import dataclass
from rl.distribution import SampledDistribution
from rl.markov_process import State, NonTerminal, Terminal
from rl.markov_decision_process import MarkovDecisionProcess
from pprint import pprint
from rl.policy import DeterministicPolicy
from rl.distribution import Constant
from itertools import islice
from rl.function_approx import LinearFunctionApprox
from rl.approximate_dynamic_programming import value_iteration
from rl.approximate_dynamic_programming import ValueFunctionApprox
from rl.iterate import converged

## Question 1: Racing Optimization (20 Points)

### Problem Setup

Let's say you are running a 400m race. You have to blend speed and stamina. If you go too fast for too long, you will run out of stamina and risk having to go too slow due to too little stamina. If you go too slow for too long, you might not be able to make up for too much time consumed. You have to find the best balance of speed and stamina conservation. Let us formalize this problem.

Let the race length be $D$ meters. Let the total stamina at the start be $S$. When your remaining stamina is $s$, you can run with a maximum effort of $e_{max}(s)$ where $e_{max}$ is a non-decreasing function, with $e_{max}(0) = 0$. To be clear, when your stamina is $s$, you make a choice to run with an effort $e$ where $0 \leq e \leq e_{max}(s)$. When you run with an effort $e$, your speed $v$ is a random variable (due to variable wind and track conditions) defined as $v = \max(0, w)$ where $w$ follows a normal distribution with mean $f(e)$ and standard deviation $\alpha \cdot f(e)$ where $f$ is a non-decreasing function, with $f(0) = 0$, and $\alpha \geq 0$ is a constant (for simplicity, assume speed is constant in each time step and that speed can be instantaneously changed from one time step to the next). When at stamina $s$, running with effort $e$ and speed $v$ for time $\Delta t$ depletes your stamina by $\beta \cdot e \cdot v \cdot \Delta t$ for some constant $\beta > 0$. Find the optimal effort-application strategy at any point in the race to minimize the total time to run this race.  

### Part 1 (6 Points)

Let us start by modeling this problem as an MDP. Let us discretize time in $\Delta t$ seconds intervals with each time step indexed by $t = 0, 1, 2, \ldots$

1. First, let's determine the state space. We want to model the states as tuples of length $2$. Model the state space. Don't forget to indicate the terminal state(s).

2. Let's next determine the possible actions. In each state, what actions can be taken?

3. What is the reward when the agent transitions from one state to the next (knowing that ultimately you want to minimze the total time to run the entire race)? *Hint: The reward is not constant across all states. You could run out of stamina or finish the race in the middle of a timestep, which creates cases for the way the reward function is represented.*

#### Answer

1. **FILL IN**
2. **FILL IN**
3. **FILL IN**

### Part 2 (6 Points)

We have provided starter code below. Your task is to fill in the `sample_next_state_reward()` and the `actions()` functions below. The class is defined in a way that makes it easy to instantiate, as you will see in future parts; as such, do not modify any code outside of what is directed here. After you modify the code in the cell below, run the next cells. Only modify code where you are told.

In [ ]:
FloatPair = Tuple[float, float]

HIGH_NEGATIVE = -1e6
TRANSITION_SAMPLES = 300

@dataclass(frozen=True)
class SpeedAndStamina(MarkovDecisionProcess[FloatPair, float]):
  # total distance to be run in the race
  D: float
  # total stamina available at the start of the race
  S: float
  # number of discrete action-choices (evenly-spaced)
  A: int
  # time-interval for each time step
  delta_t: float
  # function defining max effort, given remaining stamina
  e_max: Callable[[float], float]
  # function defining mean speed, given effort
  f: Callable[[float], float]
  # parameter alpha defining stdev of speed as a fraction of mean speed
  alpha: float
  # parameter beta defining stamina depletion, given effort and speed
  beta: float

  def step(
    self,
    state: NonTerminal[FloatPair],
    e: float
  ) -> SampledDistribution[Tuple[State[FloatPair], float]]:
    d, s = state.state
    mean = self.f(e)

    def sample_next_state_reward() -> Tuple[State[FloatPair], float]:
      next_state = _
      reward = _

      '''
      FILL IN
      '''

      return next_state, reward

    return SampledDistribution(
        sampler=sample_next_state_reward,
        expectation_samples=TRANSITION_SAMPLES
    )

  def actions(self, state: NonTerminal[FloatPair]) -> Iterator[float]:
    '''
    FILL IN
    '''
    _, s = state.state
    max_action = _ # fill this in (only modify this line)
    return ((i + 1) * max_action / self.A for i in range(self.A))

In [ ]:
def e_max(s: float) -> float:
    E_max = 40.0   # Saturation level
    s0 = 50.0      # Midpoint stamina
    k = 0.1        # Steepness
    return E_max / (1.0 + np.exp(-k * (s - s0)))

stamina_vals = np.linspace(0, 100, 101)
effort_vals = [e_max(s) for s in stamina_vals]

plt.figure(figsize=(8,6))
plt.plot(stamina_vals, effort_vals, '-o')
plt.title(r"Logistic $e_{\max}(s) = \frac{40}{1 + e^{-0.1(s - 50)}}$")
plt.xlabel("Stamina (s)")
plt.ylabel("Max Effort (e_max)")
plt.xlim(100,0)
plt.grid(True)
plt.show()

In [ ]:
'''
Now, let's create an instance of this problem, and a simple policy such that the
effort chosen is a constant fraction (call it \beta) of the maximum effort
$e_{max}(s)$ allowed at any state.
'''

D: float = 400.0
S: float = 100.0
A: int = 10
delta_t: float = 1
e_max: Callable[[float], float] = lambda s: 40.0/(1.0+np.exp(-0.1*(s-50.0)))
f: Callable[[float], float] = lambda y: y
alpha: float = 0.1
beta: float = 0.025

sas = SpeedAndStamina(
  D=D,
  S=S,
  A=A,
  delta_t=delta_t,
  e_max=e_max,
  f=f,
  alpha=alpha,
  beta=beta
)

beta = 0.25
policy_func = lambda state: beta * e_max(state[1])

dp = DeterministicPolicy(action_for=policy_func)

traces = sas.action_traces(Constant(value=NonTerminal((D, S))), dp)

num_traces = 1000

times = []
staminas = []
for trace in islice(traces, num_traces):
  elems = list(trace)
  last_d, last_s = elems[-1].next_state.state
  if last_d == 0:
      times.append(-sum(x.reward for x in elems))
      staminas.append(last_s)

print(f"There were {len(times)} finishes, average time for finishes was \
{np.mean(times):.2f} and the average finishing stamina was \
{np.mean(staminas):.2f}")

If your average time for finishes is around $80$ and your average finishing stamina is around $35$, you're on the right track! Great job!

### Part 3 (2 Points)

Next, implement the approximate value iteration algorithm. We have defined feature functions, and you only need to fill in the `done_func()` function.

Don't be worried if it takes a long time to run. Mine takes 9 minutes, and the times can vary. However, if executing this cell takes longer than 20 minutes, you probably have a mistake.

In [ ]:
NUM_STATE_SAMPLES = 300
ERROR_THRESHOLD = 0.2

ffs = [
    lambda _: 1.0,
    lambda state: state.state[0],
    lambda state: state.state[1],
    lambda state: state.state[0] * state.state[0],
    lambda state: state.state[1] * state.state[1],
    lambda state: state.state[0] * state.state[1]
]
vf_approx_init = LinearFunctionApprox.create(ffs)

uniform = SampledDistribution(
    sampler=lambda: NonTerminal((np.random.uniform(0, D), np.random.uniform(0, S)))
)

vi = value_iteration(
    mdp=sas,
    γ=1,
    approx_0=vf_approx_init,
    non_terminal_states_distribution=uniform,
    num_state_samples=NUM_STATE_SAMPLES
)

def done_func(fa1: ValueFunctionApprox[FloatPair], fa2: ValueFunctionApprox[FloatPair]) -> bool:
  distances = np.linspace(0, D, 10)
  staminas = np.linspace(0, S, 10)
  '''
  FILL IN
  Replace the underscore with your code
  '''
  return max(_) < ERROR_THRESHOLD

opt_vf = converged(vi, done=done_func)

### Part 4 (2 Points)

Now let's extract the Optimal Policy from this Optimal Value Function. Fill in the code below.

In [ ]:
def opt_policy(state: FloatPair) -> float:
  def q_value(e: float) -> float:
    '''
    FILL IN
    '''
    return _
  return max(sas.actions(NonTerminal(state)), key=q_value)

policy = DeterministicPolicy(action_for=opt_policy)

### Part 5 (1 Point)

Run the following cell to generate a heatmap and comment on the heatmap. Does the color grading make sense? Is there anything that doesn't make intuitive sense? If so, comment on why this might be the case.

In [ ]:
x_vals = np.linspace(0, D, 10)
y_vals = np.linspace(0, S, 10)
Y, X = np.meshgrid(y_vals, x_vals)
all_times = [[-opt_vf(NonTerminal((d, s))) for s in y_vals] for d in x_vals]
plt.figure(figsize=(8, 6))
plt.pcolormesh(X, Y, all_times, shading='auto', cmap='viridis')
plt.colorbar(label="Time Taken")
plt.xlabel("Distance Left")
plt.ylabel("Stamina Left")
plt.title("Optimal Time Taken as a function of distance and stamina left")
plt.show()

#### Answer

**FILL IN**

### Part 6 (1 Point)

Explain the impact of feature functions. What do they do? Why are they relevant in approximate value iteration, and why are the relevant in this problem? What would happen if we added more feature functions?

#### Answer

**FILL IN**

### Part 7 (1 Point)

Run the code below and comment on the graphs for the varying stamina levels.

In [ ]:
stamina_levels = [0, 10, 25, 50, 75, 100]
distance_values = np.linspace(0, D, 50)
plt.figure(figsize=(8, 6))
for s_fixed in stamina_levels:
    speeds = []
    for d_val in distance_values:
        e = policy.act(NonTerminal((d_val, s_fixed))).value
        speed = sas.f(e)
        speeds.append(speed)
    plt.plot(distance_values, speeds, label=f"Stamina = {s_fixed}")

plt.xlabel("Distance Left")
plt.ylabel("Speed (mean)")
plt.title("Agent Speed vs. Distance Left for Various Stamina Levels")
plt.legend()
plt.show()

#### Answer

**FILL IN**

### Part 8 (1 Point)

For the final part of this problem, we want to think about different ways to solve this problem using RL. Which RL algorithm do you think is best suited to solve this problem? Why? A few, detailed sentences is fine.

#### Answer

**FILL IN**

## Question 2: Let's Play a Game! (10 Points)

### Problem Setup

You want to design a profitable and sticky quiz game. It's you asking questions versus the players answering questions. You are thinking of 2 games.

**Game 1**: To play a round, a player has to pay you \\$10. There are 10 questions in a round. If a player answers 1 question correctly, he wins \\$1.  But if he gets 7, 8 or 9 questions correct, he wins an extra \\$10 bonus. For example, if he gets 8 answers correct, he wins 8 + 10 = \\$18, if he answers 9 answers correctly, he wins 9+10= \\$19. But if he answers all 10 questions correctly, he wins \\$100.

Also, if a player wins at least \\$5 or more from a round, he will play one more time. If he wins less than \\$5 from a round, he gets eliminated. The game ends as soon as the player gets eliminated. Note that there is no other termination condition for this game other than a round where the player wins less than \$5.

**Game 2**: A player pays you \$10 to play each round. There are 10 questions in a round. If a player answers at least 8 of the 10 questions right, he wins $5^2=25$ dollars and goes to the second round. Otherwise (if he answers less than 8 questions right), he gets eliminated. If he answers at least 8 questions right in the second round, he wins $5^3=125$ dollars and goes to the third round. Otherwise (if he answers less than 8 questions right), he gets eliminated. If he answers at least 8 questions right in the third round, he wins $5^4=625$ dollars and goes to the fourth round. Likewise, he wins $5^5=3125$ in round four if he answers at least 8 questions right (basically, the prize money multiplies 5 times in each round). The game ends as soon as the player gets eliminated or after 4 rounds are completed.

If a large number of players play this game, how much money are you expected to make in each of the two games, assuming that the probability of a random player answering any single question correctly is 50% (in any round, in each of the two games)?

Which of these two games is more profitable for you (on an expected basis)?

Hint: You can model these two games as Markov Reward Processes (MRPs), and evaluate the Value Function for these MRPs.

### Part 1 (1 Point)

Let $p_i$, where $0 \leq i \leq 10$, denote the probability of a random player answering exactly $i$ questions correctly in any round. What is $p_i$ equal to? Note that we've assumed that the probability of answering each question correctly is 50%.

#### Answer

**FILL IN**

### Part 2 (2 Points)

For Game 1, let us denote $R_i$ as the money you have to give to a player in any round when the player answers $i$ questions correctly. We want to define an infinite-horizon MRP (in terms of $R_i$ and $p_i$, $0 \leq i \leq 10$), whose Value Function gives us the Expected Gains for you in Game 1.

1. What is the state space and the state-transition probability function of this infinite-horizon MRP?
2. What is the reward function?
3. Write the Bellman Equation for this MRP.
4. Solve for the Value Function.

#### Answer

1. **FILL IN**
2. **FILL IN**
3. **FILL IN**
4. **FILL IN**

We have provided starter code for you to use to evaluate the Value Function you solved above. Fill in the V_game1 expression to solve for the expected gains.

In [ ]:
# q: probability of getting a question right
q = 0.5 # assume equal probability of right or wrong answer
# p: probability of player answering exactly i questions correct in any single
# round
'''
Define p as an array using list comprehension so that we have
probabilities for all values of i; fill in the list comprehension with your
previously derived mathematical expression. This might be helpful:
https://www.w3schools.com/python/ref_math_comb.asp
Only fill in the underscore with code; keep the list comprehension as is
'''
p = [_ for i in range(11)]
# R: list of prize amounts the player recieves for each possible number of
# correct answers from 0 to 10
'''
This list should be hardcoded and should be of length 11; define it such
that R[0] is the prize if the player gets 0 correct, R[8] is the prize if the
player gets 8 correct, etc.
'''
R = [0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 100]
# V1: the expected gains for you (from each player) in Game 1
V_game1 = _
print(f"Expected Gains for you (from each player) in Game 1 is {V_game1:.2f}")

### Part 3 (2 Points)
We want to generalize Game 2 such that:

-  Instead of winning $5^{j+1}$ in round $j$, the player wins $c^{j+1}$ for some arbitrary positive real number $c$.
- Instead of needing to answer a minimum of 8 questions right to get to the next round (and winning a reward), the player needs to answer a minimum of $m$ questions right, for some arbitrary integer $0 \leq m \leq 10$.
- Instead of the game terminating after 4 rounds, the game terminates after $n$ rounds, for some arbitrary integer $n \geq 1$.

Unlike Game 1, Game 2 is a finite-horizon MRP. Here our state space is $\{1, 2, \ldots, n\}$. The state at the start of round $j$ is $j$, for all $1 \leq j \leq n$. Let $V_j$ denote the Value Function for state $j$.

As we have always handled with finite-horizon problems, you can work your way backwards in time (backward induction method for solving the Value Function of a finite-horizon MRP). Start by working out the expression for $V_n$ (note: the game terminates after the $n$-th round, no matter how many questions are answered in round $n$). Next, write $V_j$ in terms of $V_{j+1}$, $p_i$ and $c$, for all $1 \leq j \leq n-1$ ($p_i$ is the probability that the player gets $i$ questions right in any round). This is essentially the MRP Bellman Equation. To work out the MRP Bellman Equation (just like in Game 1), you need the state transition probability function and the reward function. But you have to be careful in taking into account premature termination in each round (if the player answers less than $m$ questions right). It will be useful for you to define $P_m = \sum_{i=m}^{10} p_i$, which is the probability of moving to the next round.



Fill in the following two expressions:

#### Answer

1. $V_n = \_$
2. $V_j=\_ \mbox{ for all } 1 \leq j \leq n - 1$ (MRP Bellman Equation)

### Part 4 (3 Points)

Using the MRP Bellman Equation you worked out above, derive a closed-form expression for $V_j$. To achieve this, do repeated substitutions, or use induction, to eliminate $V_{j+1}, V_{j+2}, \dots$. You should end up with a summation that explicitly depends on $j,n,P_m$, and $c$. In your final answer, show $V_j$ purely as a function of these parameters without any remaining recursion.

**Hint: Unroll or Use a Pattern**

   Once you have a recursive relationship—something like  
   $$V_j = (\text{some constant}) + (\text{some probability}) \times (\text{next } V_{j+1}) - \dotsb$$  
   you can either  
   1. Do *repeated substitution*, eliminating $V_{j+1}, V_{j+2}, \dots$ until you reach $V_n$.  
   2. Or *guess* a summation expression for $V_j$ (involving powers of the pass probability and the prize cost) and then prove by induction that it satisfies your recursion.  

   Look for a *geometric-series*-like pattern involving the pass probability each time the player advances. Be methodical in tracking how many times you subtract the cost $c$-term and how many times you still collect \$10.

#### Answer

1. $V_j=\_ \mbox{ for all } 1 \leq j \leq n$

Note that the final answer you are seeking for Game 2 is $V_1$

### Part 5 (1 Point)

We define parameters below. Fill in the code below to implement the value function you solved above.

In [ ]:
c = 5
m = 8
n = 4

Pm = sum(p[m:])
'''
FILL IN
'''
V_game2 = _
print(f"Expected Gains for you (from each player) in Game 2 is {V_game2:.2f}")

### Part 6 (1 Point)

Which game would you rather play? Why?

#### Answer

**FILL IN**